Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-21 20:16:08--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-21 20:16:09--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   115KB/s    in 1.8s    

2021-10-21 20:16:12 (115 KB/s) - ‘data.zip’ saved [217313/217313]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [4]:
import os
import time
import random
import spacy
import dill
from tqdm import tqdm
import datetime
import numpy as np
import pandas
import logging
from argparse import ArgumentParser
from pdb import set_trace
import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments

In [6]:
## Various utility functions
class args_class:
    dataset = "data/"
    model_name = "bert-base-uncased"
    gpu = 0
    batch_size = 8
    results_dir = "2018EE10957_B_model"

In [7]:
def set_seed(seed=4):
	# fixing seeds to reproduce results (exact!)
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)

def parse_args():
	parser = ArgumentParser(description='NLP A3-A')
	parser.add_argument('--dataset', '-d', type=str, default='data')
	parser.add_argument('--model_name', '-m', type=str, default='bert-base-uncased')
	parser.add_argument('--gpu', type=int, default=0)
	parser.add_argument('--batch_size', type=int, default=8)
	# parser.add_argument('--hidden_dim', type=int, default=128)
	# parser.add_argument('--epochs', type=int, default=20)
	# parser.add_argument('--lr', type=float, default=0.001)
	parser.add_argument('--results_dir', type=str, default='2018EE10957_B_model')
	return parser.parse_args()

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
		filename = "{}_{}.log".format(args.model, phase),
		format = '%(asctime)s - %(message)s', 
		datefmt='%d-%b-%y %H:%M:%S'
	)
	return logging.getLogger(phase)

In [8]:
def process_data(path, train=True):
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

class myDataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, model_name='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
            max_length=self.max_len, padding='max_length', truncation=True)
        y = torch.tensor(sample['label'])

        # format as required by Train
        item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
        item['labels'] = y
        return item
    
    def __len__(self):
        return len(self.dataset_df)

In [9]:
# dataset_df = process_data("data", train=False)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
# dataset_df.iloc[12:14]['sen1'].values
# sample = dataset_df.iloc[12:14]
# x = tokenizer(list(sample['sen1']), list(sample['sen2']), return_tensors='pt', \
#                         add_special_tokens=True, max_length=128, padding='max_length')
# y = torch.tensor(list(sample['label']))
# for k,v in x.items():
#     print(v.shape)

In [10]:
set_seed()

class Train():
    def __init__(self):
        print("program execution start: {}".format(datetime.datetime.now()))
        self.args = args_class()
        self.device = get_device(self.args.gpu)
        # self.logger = get_logger(self.args, "train")
        # self.logger.info("Arguments: {}".format(self.args))

        self.trainset = myDataset(self.args.dataset, max_len=100, train=True, model_name=self.args.model_name)
        self.devset = myDataset(self.args.dataset, max_len=100, train=False, model_name=self.args.model_name)

        # self.trainloader = DataLoader(trainset, batch_size = args.batch_size, shuffle = True)
        # self.devloader = DataLoader(devset, batch_size = args.batch_size, shuffle = False)

        # self.model = myModel(self.device)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.args.model_name, num_labels=2)

        # self.model = self.model.to(self.device)
        # self.criterion = nn.BCELoss()
        # self.opt = O.Adam(self.model.parameters(), lr = self.args.lr) #, weight_decay=0.005)
        # self.best_val_acc = None
        # self.scheduler = StepLR(self.opt, step_size=5, gamma=0.5)

        print("resource preparation done: {}".format(datetime.datetime.now()))

    def execute(self):
        # some parameters taken from https://huggingface.co/transformers/training.html
        training_args = TrainingArguments(
            output_dir= self.args.results_dir, 
            overwrite_output_dir=True, 
            evaluation_strategy="epoch", 
            learning_rate=5e-5,
            weight_decay=0.01,
            num_train_epochs=3, 
            seed=4,
            per_device_eval_batch_size=self.args.batch_size,
            per_device_train_batch_size=self.args.batch_size,
            warmup_steps=500,
            logging_dir='./logs',
            logging_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="eval_accuracy",
            greater_is_better=True,
        )

        trainer = Trainer(
            model=self.model, args=training_args,
            train_dataset=self.trainset, eval_dataset=self.devset,
            compute_metrics=compute_metrics# deplag
        )

        trainer.train()

In [11]:
# import numpy as np
# from datasets import load_metric

# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [20]:
# metric

In [12]:
# import transformers
# transformers.logging.set_verbosity(40)

In [13]:
## Start training
task = Train()
task.execute()

program execution start: 2021-10-24 14:27:32.931952


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

resource preparation done: 2021-10-24 14:27:55.225020


In [19]:
training_args = TrainingArguments(
    output_dir= task.args.results_dir, 
    overwrite_output_dir=True, 
    evaluation_strategy="epoch", 
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=4, 
    seed=4,
    per_device_eval_batch_size=task.args.batch_size,
    per_device_train_batch_size=task.args.batch_size,
    warmup_steps=500,
    logging_dir='./logs',
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
)

trainer = Trainer(
    model=task.model, args=training_args,
    train_dataset=task.trainset, eval_dataset=task.devset,
    compute_metrics=compute_metrics# deplag
)

m=trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 5428
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 456
/home/shubham/anaconda3/envs/nlp/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.095100,1.146554,0.677116
2,0.046500,1.725659,0.666144
3,0.038700,2.072562,0.655172
4,0.136500,1.395411,0.670846


***** Running Evaluation *****
  Num examples = 638
  Batch size = 48
Saving model checkpoint to 2018EE10957_B_model/checkpoint-114
Configuration saved in 2018EE10957_B_model/checkpoint-114/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-114/pytorch_model.bin
/home/shubham/anaconda3/envs/nlp/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 638
  Batch size = 48
Saving model checkpoint to 2018EE10957_B_model/checkpoint-228
Configuration saved in 2018EE10957_B_model/checkpoint-228/config.json
Model weights saved in 2018EE10957_B_model/checkpoint-228/pytorch_model.bin
/home/shubham/anaconda3/envs/nlp/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, 

In [16]:
# m

TrainOutput(global_step=114, training_loss=0.6710249917548999, metrics={'train_runtime': 48.3189, 'train_samples_per_second': 112.337, 'train_steps_per_second': 2.359, 'total_flos': 278938829784000.0, 'train_loss': 0.6710249917548999, 'epoch': 1.0})

In [ ]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018EE10957_A_model.zip 2018EE10957_B_model

## Upload it to Google drive and ensure that the testing notebook uses the correct link

## debugging

In [58]:
# training_args = TrainingArguments(
#     output_dir=task.args.results_dir, 
#     overwrite_output_dir=True, 
#     evaluation_strategy="epoch", 
#     learning_rate=5e-5,
#     num_train_epochs=3, 
#     seed=4,
#     per_device_eval_batch_size=task.args.batch_size,
#     per_device_train_batch_size=task.args.batch_size,
# )

# trainer = Trainer(
#     model=task.model, args=training_args,
#     train_dataset=task.trainset, eval_dataset=task.devset,
# )

# trainer.train()

In [1]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.mask_token_id)
# tokenizer.encode(["hey my name is shubham", "they 'll play this game very hastily"], add_special_tokens=True)
# print(tokenizer.convert_tokens_to_ids(["[PAD]", "[SEP]", "[CLS]", "[MASK]"]))
# print(tokenizer.convert_ids_to_tokens([0,100,101, 102, 103, 104]))

In [3]:
# df = process_data("./data/", train=False)
# df.iloc[0]

In [19]:
# out = tokenizer(["heyhey myhey","hi","hiello"], add_special_tokens=False, return_tensors='pt', padding='max_length', max_length=5, truncation=True)
# out

In [10]:
# tokens = tokenizer.convert_ids_to_tokens(out['input_ids'][0])
# print(tokens)
# sen = tokenizer.convert_tokens_to_string(tokens)
# print(sen)
# print(" ".join(tokens))

In [20]:
# from transformers import AutoModelForSequenceClassification
# bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# bert = bert.to('cuda')
# trainset = WiC_dataset("data", max_len=128, train=True, tokenizer='bert-base-uncased')
# trainloader = DataLoader(trainset, batch_size = 32, shuffle = True)
# out = bert(x)

In [26]:
# df = process_data("./data/", train=True)
# max_num_tok = 0
# for i in range(len(df)):
#     t = tokenizer(df.loc[i,'sen1'], df.loc[i,'sen2'], add_special_tokens=True, return_tensors='pt')['input_ids']
#     max_num_tok = max(max_num_tok, t.shape[-1])
# max_num_tok

69

In [29]:
# tokens = tokenizer.convert_ids_to_tokens(t[0])
# print(tokens)
# sen = tokenizer.convert_tokens_to_string(tokens)
# print(sen)
# print(" ".join(tokens))
# tokenizer.decode(t[0])

'[CLS] an invasion of bees. [SEP] an invasion of mobile phones. [SEP]'

In [5]:
def process_data(path, train=True):
    train = "train" if train else "validation"
    x_train = pandas.read_csv(os.path.join(path, "%s/%s.data.txt"%(train, train)), delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    y_train = pandas.read_csv(os.path.join(path, "%s/%s.gold.txt"%(train, train)), header=None, names=['label'])
    dataset = pandas.concat((x_train, y_train), axis=1)
    word1 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = dataset.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    dataset = dataset.drop("position", axis=1)
    dataset.loc[:, "POS"] = dataset.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    dataset.loc[:, "label"] = dataset.loc[:, "label"].apply(lambda lab : 0 if lab=="F" else 1)
    dataset.insert(3, "word1", word1)
    dataset.insert(5, "word2", word2)
    # dataset.to_csv(os.path.join(path, "%s/%s.csv"%(train, train)), header=None, index=False)
    return dataset

class myDataset(Dataset):
    def __init__(self, data_dir, max_len=128, train=True, model_name='bert-base-uncased'):
        self.data_dir = data_dir
        self.max_len = max_len
        self.train = train

        # convert raw data into pandas data frame
        self.dataset_df = process_data(data_dir, train=train)

        # tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

    def __getitem__(self, index):
        sample = self.dataset_df.iloc[index]

        x = self.tokenizer(sample['sen1'], sample['sen2'], return_tensors='pt', add_special_tokens=True, \
            max_length=self.max_len, padding='max_length', truncation=True)
        y = torch.tensor(sample['label'])

        # format as required by Train
        item = {k: v[0] for k,v in x.items()} # only one datapoint; shape: 128
        item['labels'] = y
        return item
    
    def __len__(self):
        return len(self.dataset_df)

In [14]:
from transformers import AutoModel
bert = AutoModel.from_pretrained("bert-base-cased", num_labels=2)
bert = bert.to(torch.device('cuda'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
trainset = myDataset("data", max_len=100, train=True, model_name="bert-base-cased")
devset = myDataset("data", max_len=100, train=False, model_name="bert-base-cased")

trainloader = DataLoader(trainset, batch_size = 8, shuffle = True)
devloader = DataLoader(devset, batch_size = 8, shuffle = False)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
batch=next(iter(trainloader))

In [15]:
inp={k:v.to(torch.device('cuda')) for k,v in batch.items() if k != 'labels'}
oup = bert(**inp)

In [23]:
print(len(oup), oup[0].shape, oup[1].shape)

2 torch.Size([8, 100, 768]) torch.Size([8, 768])


In [26]:
for p in bert.parameters():
    p.requires_grad=False

## testing

In [ ]:
def main():
    hidden_dim=64
    device = get_device(0)

    x_test = pandas.read_csv("test.data.txt", delimiter='\t', header=None, names=['word', 'POS', 'position', 'sen1', 'sen2'])
    word1 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[0])
    word2 = x_test.loc[:,'position'].apply(lambda positions : positions.split("-")[1])
    x_test = x_test.drop("position", axis=1)
    x_test.loc[:, "POS"] = x_test.loc[:, "POS"].apply(lambda POS_TAG : 0 if POS_TAG=="V" else 1)
    x_test.insert(3, "word1", word1)
    x_test.insert(5, "word2", word2)
    x_test.to_csv("./test.csv", header=None, index=False)

    with open(os.path.join("2018EE10957_A_model", "Field_TEXT"), 'rb') as f:
        TEXT = dill.load(f)

    fields = [
        ('word', TEXT),
        ('POS', data.Field(use_vocab=False, sequential=False)), 
        ('sen1', TEXT),
        ('word1', data.Field(use_vocab=False, sequential=False)),
        ('sen2', TEXT),
        ('word2', data.Field(use_vocab=False, sequential=False)),
    ]

    test_set = data.TabularDataset(
        path = "./test.csv",
        format = 'csv',
        fields = fields,
        skip_header = False
    )

    test_itr = data.BucketIterator(
        test_set,
        #sort_key = lambda sample : len(sample.sen1),
        sort = False,
        batch_size = 32,
        repeat = False,
        shuffle = False,
    )

    model = myModel(pretrained_embeddings=TEXT.vocab.vectors, embed_dim=TEXT.vocab.vectors.shape[1], hidden_dim=hidden_dim, device=device)
    saved_model = torch.load("2018EE10957_A_model/best-biLSTM-params.pt", map_location=device)
    print(saved_model["accuracy"])
    model.load_state_dict(saved_model['model_dict'])
    model = model.to(device)

    model.eval()
    test_itr.init_epoch()
    predictions = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_itr):
            answer = model(batch)
            answer = list(np.array((answer>=0.5).long().cpu()))
            predictions += answer

    predictions = pandas.DataFrame(predictions)[0].apply(lambda label : "T" if label == 1 else "F")
    predictions.to_csv("output.txt", header=None, index=None)
    print("predictions written in output.txt")

In [ ]:
main()

60.18808777429467
predictions written in output.txt
